# Dependencies, functions, initializations

In [ ]:
 # dependencies

import os
import math
import scipy
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import itertools
import random
import json
import seaborn as sns
import pickle

from numpy import isnan
from random import shuffle

from scipy.signal import find_peaks
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt

from bokeh.plotting import figure, show, output_file, save
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource, DatetimeTickFormatter, CheckboxGroup, CustomJS, Title, Span, LabelSet
from bokeh.palettes import Category20 as palette
from bokeh.colors import RGB
from bokeh.embed import json_item


In [ ]:
output_notebook()

Loading BokehJS ...

In [ ]:
# functions

def rms_signal(data, window_size, scroll):
  i = int(window_size/2)
  l = data.shape[0]
  rms_y = [np.nan]*l
  while True:
    if i >= l:
      break

    if (i >= window_size/2) and (l-i >= window_size/2):
      in_window  = int(i-window_size/2)
      out_window = int(i+window_size/2)
      data_rms = data.iloc[in_window:out_window]
      rms_value = math.sqrt(sum([i*i for i in data_rms])/len(data_rms)) if data_rms.isna().sum() == 0 else np.nan
    else:
      rms_value = np.nan
    
    rms_y[i] = rms_value
    #[rms_y.append(np.nan) for i in range(int(scroll-1))]
    i = i + scroll

    # crop nans
    rms_y = rms_y[:data.shape[0]]

  df = pd.DataFrame(index = data.index, columns = ['rms'], data={'rms': rms_y})
  return df

def butter_lowpass_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff / nyq
  # get coefficients
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y

"""
# rms
def rms_(data):
  return math.sqrt(sum([i*i for i in data])/len(data))
# hard limiter
def hard_limiter(data, column_name, limit):
  data.loc[data[column_name] > limit, column_name] = limit
  return data
"""

# FinnPy

def nannotime(row,shift,dshift):
    r = row.copy()
    cols = r.index
    for c in cols:
        if not c.startswith('time'):
            if not c.startswith('datetime'):
                r[c] = np.nan
            if c.startswith('datetime'):
                r[c] = r[c] + dshift  
        if c.startswith('time'):
            r[c] = r[c] + shift     
    return r

def gap_nans(data,gap_t):
    # data is a pandas dataframe with columns called time and timestamps which is used to ID gaps greater than gap_T
    # rows of NaN data is added to non-time columns before the first sample, in each gap, and after the last sample of data
    cols = data.columns
    if 'time' in cols:
        time_col = 'time'
    if 'time_concert' in cols:
        time_col = 'time_concert'
    deltat = round(0.35*data[time_col].diff().median())
    dtdeltat =  pd.to_timedelta(deltat,unit = 'ms')
    
    data = data.append(nannotime(data.iloc[-1,:],deltat,dtdeltat),ignore_index=True)
    
    dt = data[time_col].diff()
    a = list(dt[dt>gap_t].index)
    a.sort(reverse=True)
    for gapi in a:
        data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
        data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))

    data = data.append(nannotime(data.iloc[0,:],-1,dtdeltat)).sort_values(time_col,ignore_index=True)
    
    return data

# Plot stats for energy

In [ ]:
# plot rms stats

sample_rate = 50 # Hz

dir_events = 'C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/'
f_clappings_with_tags = 'C:/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_clean_tags.csv' 
output_dir = 'C:/ritmo/CopenhagenMusicLab/Concert_content/other_plot/'

events = sorted([dir_events + ev for ev in os.listdir(dir_events) if ev.endswith('.csv')])
df_clappings = pd.read_csv(f_clappings_with_tags)

"""
sum_rms_x = []
sum_rms_y = []
for ev in events[:1]:
  df_ev = pd.read_csv(ev, index_col=0)
  data = pd.DataFrame(index = df_ev.index)
  for col in df_ev.columns:
    data[col] = rms_signal(df_ev[col], sample_rate, sample_rate)
  sum_rms_y.append(data.mean(axis=1).dropna().sum())

"""

data = pd.DataFrame()
for ev in events:
  print(ev)
  rms_list = []
  df_ev = pd.read_csv(ev, index_col=0)
  for col in df_ev.columns:
    rms_list.append(rms_signal(df_ev[col], sample_rate, sample_rate).mean().values[0])
    #rms_list.append(df_ev[col].shape[0])
  print(len(rms_list))
  data[ev.split('_')[-2].split('.')[0]] = rms_list

data = data.T.reset_index(drop=True)
data.index = data.index.astype(str)

labels_list = df_clappings.tag.values

p = figure(title = 'Total energy clapping sessions', x_range=labels_list, plot_width=1250, plot_height=1200, sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')

for col, c in zip(data.columns, palette[20]*10):
  #p.varea(x=labels_list, y1=0, y2=data[col], fill_color=c, fill_alpha=0.1, legend_label='Devices')
  #p.line(x=labels_list, y=data[col], line_color=c, line_alpha=0.1, legend_label='Devices')
  p.scatter(x=labels_list, y=data[col], fill_color=c, hatch_color=c, line_color=c, fill_alpha=0.7, legend_label='Devices')

p.vbar(x=labels_list, top=data.mean(axis=1), width = 0.75, fill_alpha = 0.4, legend_label='Average') #, line_color=palette[20][2], line_width=3)

#p.xaxis.ticker = df_clappings.tag.to_list()
p.xaxis.major_label_overrides = dict(zip(range(data.shape[0]), df_clappings.tag.iloc[:data.shape[0]]))
#p.xaxis.ticker.desired_num_ticks = data.shape[0]
p.xaxis.major_label_orientation = 3*math.pi/8
p.xaxis.major_label_text_font_size = '14pt'
p.legend.click_policy = 'hide'
p.title.text_font_size = '20pt' #############

p.add_layout(Title(text='Energy', align='center'), 'left')

output_file_name = 'stats.html' #p.properties_with_values()['title'].text.replace(' ', '_') + '.html'
output_file(output_dir + output_file_name)

save(p)

"""
list_of_list_x = [data.index for i in range(df_ev.shape[0])]
list_of_list_y = [data.iloc[i].to_list() for i in range(df_ev.shape[0])]
list_labels = [str(i) for i in range(df_ev.shape[0])]

data_bokeh = pd.DataFrame()
data_bokeh['x'] = list_of_list_x
data_bokeh['y'] = list_of_list_y
data_bokeh['color'] = (palette[20]*100)[:len(data)]
data_bokeh['labels'] = list_labels


p = figure(title = 'Total RMS for clapping ' + ev.split('_')[-1].split('.')[0], plot_width=1250, plot_height=300, sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
p.add_layout(Title(text='Clappings', align='center'), 'below')
p.add_layout(Title(text='Values', align='center'), 'left')
p.legend.location = 'above'
p.multi_line(xs='x', ys='y', source=data_bokeh, color='color', legend='labels')
p.xaxis.ticker = np.arange(df_ev.shape[0])
p.legend.click_policy="hide"
show(p)
"""

0

C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/01_When_musicians_appear,_before_the_introduction___947.0_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/02_Between_Frederik_&_Simon_talks___1082.19_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/03_After_the_introduction,_before_the_concert_begins___1669.07_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/04_Beethoven___3249.6_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/05_Musicians_back_to_stage_for_Schnittke___3642.69_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/06_Schnittke___4921.6_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/07_Musicians_back_after_intermission___6640.590000000002_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/08_Bach___7658.689999999999_sec.csv
75
C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/09_Folk_M

0

# Plot stats for duration

In [ ]:
# plot stats duration

sample_rate = 50 # Hz

dir_events = 'C:/ritmo/old/CopenhagenMusicLab_thrown-14-12/Concert_content/clapping_sections/'
f_clappings_with_tags = 'C:/ritmo/old/CopenhagenMusicLab_thrown-14-12/Concert_content/Concert_event_times_clean_tags.csv' 
output_dir = 'C:/ritmo/old/CopenhagenMusicLab_thrown-14-12/Concert_content/other_plot/'

events = sorted([dir_events + ev for ev in os.listdir(dir_events) if ev.endswith('.csv')])
df_clappings = pd.read_csv(f_clappings_with_tags)

"""
sum_rms_x = []
sum_rms_y = []
for ev in events[:1]:
  df_ev = pd.read_csv(ev, index_col=0)
  data = pd.DataFrame(index = df_ev.index)
  for col in df_ev.columns:
    data[col] = rms_signal(df_ev[col], sample_rate, sample_rate)
  sum_rms_y.append(data.mean(axis=1).dropna().sum())

"""

data = pd.DataFrame()
for ev in events:
  rms_list = []
  df_ev = pd.read_csv(ev, index_col=0)
  for col in df_ev.columns:
    #rms_list.append(rms_signal(df_ev[col], sample_rate, sample_rate).mean().values[0])
    rms_list.append(df_ev[col].shape[0]/50/60) # min
  data[ev.split('_')[-2].split('.')[0]] = rms_list

data = data.T.reset_index(drop=True)
data.index = data.index.astype(str)

labels_list = df_clappings.tag.values

p = figure(title = 'Duration clapping sessions', x_range=labels_list,
           plot_width=1250, plot_height=1200, sizing_mode='stretch_width',
           tools='pan,wheel_zoom,box_zoom,reset')

#for col, c in zip(data.columns, palette[20]*10):
  #p.varea(x=labels_list, y1=0, y2=data[col], fill_color=c, fill_alpha=0.1, legend_label='Devices')
  #p.line(x=labels_list, y=data[col], line_color=c, line_alpha=0.1, legend_label='Devices')
  #p.scatter(x=labels_list, y=data[col], fill_color=c, hatch_color=c, line_color=c, fill_alpha=0.7, legend_label='Devices')


lis_1 = [3,5,7,8,9,10,11,12]
lis_2 = [0,1,2,4,6,13]
lis_3 = [14]

p.vbar(x=[labels_list[i] for i in lis_2], top=data.iloc[lis_2,:].mean(axis=1), width = 0.75, fill_alpha = 0.4, fill_color='red', line_color='red', legend_label='Pieces') #, line_color=palette[20][2], line_width=3)
p.vbar(x=[labels_list[i] for i in lis_1], top=data.iloc[lis_1,:].mean(axis=1), width = 0.75, fill_alpha = 0.4, fill_color='olive', line_color='olive', legend_label='Other Claps') #, line_color=palette[20][2], line_width=3)
p.vbar(x=[labels_list[i] for i in lis_3], top=data.iloc[lis_3,:].mean(axis=1), width = 0.75, fill_alpha = 0.4, fill_color='grey', line_color='grey', legend_label='Final') #, line_color=palette[20][2], line_width=3)
#p.vbar(x=labels_list, top=data.mean(axis=1), width = 0.75, fill_alpha = 0.4, fill_color='olive', legend_label='Sessions') #, line_color=palette[20][2], line_width=3)

#p.xaxis.ticker = df_clappings.tag.to_list()
p.xaxis.major_label_overrides = dict(zip(range(data.shape[0]), df_clappings.tag.iloc[:data.shape[0]]))
#p.xaxis.ticker.desired_num_ticks = data.shape[0]
p.xaxis.major_label_orientation = 3*math.pi/8
p.xaxis.major_label_text_font_size = '14pt'
p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
p.title.text_font_size = '20pt' #############

#p.add_layout(Title(text='Events', align='center'), 'below')

output_file_name = 'stats_dur.html' #p.properties_with_values()['title'].text.replace(' ', '_') + '.html'
output_file(output_dir + output_file_name)

source = ColumnDataSource(data=dict(height=data.mean(axis=1).values,
                                    weight=labels_list,
                                    names=[(str(round(i,2)) + ' mins') for i in data.mean(axis=1).values]))
labels = LabelSet(x='weight', y='height', text='names',
              x_offset=5, y_offset=5, source=source,
              render_mode='canvas', text_align='center', text_font_style='normal', text_font='helvetica')


p.add_layout(labels)
p.add_layout(Title(text='Minute', align='center'), 'left')

save(p)

"""
list_of_list_x = [data.index for i in range(df_ev.shape[0])]
list_of_list_y = [data.iloc[i].to_list() for i in range(df_ev.shape[0])]
list_labels = [str(i) for i in range(df_ev.shape[0])]

data_bokeh = pd.DataFrame()
data_bokeh['x'] = list_of_list_x
data_bokeh['y'] = list_of_list_y
data_bokeh['color'] = (palette[20]*100)[:len(data)]
data_bokeh['labels'] = list_labels


p = figure(title = 'Total RMS for clapping ' + ev.split('_')[-1].split('.')[0], plot_width=1250, plot_height=300, sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
p.add_layout(Title(text='Clappings', align='center'), 'below')
p.add_layout(Title(text='Values', align='center'), 'left')
p.legend.location = 'above'
p.multi_line(xs='x', ys='y', source=data_bokeh, color='color', legend='labels')
p.xaxis.ticker = np.arange(df_ev.shape[0])
p.legend.click_policy="hide"
show(p)
"""

data.iloc[:,[3,5,7,8,9,10,11,12]]

# For each clapping section create csv files and plots

In [ ]:
# create csv files for each clapping sections

Start_time = '2021-10-26 17:30:00+0000'
End_time   = '2021-10-26 20:30:00+0000'

sample_rate= 50 # Hz

# set standard timestamps in date time and millisecond floats. Datetime for ploting, ms for interp
# 50 Hz
ts_dt = pd.date_range(pd.to_datetime(Start_time), pd.to_datetime(End_time), freq='20ms')
ts_ts = np.arange(pd.to_datetime(Start_time).timestamp(), pd.to_datetime(End_time).timestamp(), 0.02)*1000

dir_devices = 'C:/ritmo/CopenhagenMusicLab/Aligned/Hall/'
f_events = 'C:/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_clean.csv'
f_clappings = 'C:/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_clean_tags.csv'

# read events data
df_events = pd.read_csv(f_events)
df_clappings = pd.read_csv(f_clappings)
shift_concert_time = df_events[df_events.time_concert == 0].time.values[0]

# devices
n_devices = len(os.listdir(dir_devices))

index_devices = slice(4, 5) #n_devices) # which devices
exclude_devices = [3, 9] # device indexes to exclude

mask = np.zeros((n_devices), dtype='bool')
mask[index_devices] = True
mask[exclude_devices] = False

list_devices = np.array(os.listdir(dir_devices))[mask]

print(len(list_devices), ' devices.')

# events
index_events = slice(0, df_clappings.shape[0]) # which events

clapping_events_in  = df_clappings.start_time_concert.to_list()[index_events]
clapping_events_out = df_clappings.end_time_concert.to_list()[index_events]
clapping_events_types = df_clappings.clapping_type.to_list()[index_events]
clapping_events_tags  = df_clappings.tag.to_list()[index_events]

# extend window
extend_dict = {1:1, 2:2, 3:2, 4:7}

plots = []
section_n = 0

# for every event:
# go over all devices
for ev_in, ev_out, ev_type, ev_tag in zip(clapping_events_in, clapping_events_out, clapping_events_types, clapping_events_tags):
  extend = extend_dict[ev_type]
  in_event_time  = (ev_in - extend) * 10**3
  out_event_time = (ev_out + extend) * 10**3

  list_of_lists_x = []
  list_of_lists_y = []
  list_of_device_names = []

  print('time interval: ', in_event_time, out_event_time)

  dev_n = 0
  ts_ev = pd.date_range(start=pd.to_datetime(0), end=pd.to_datetime(abs(out_event_time - in_event_time), unit='ms'), freq='20ms')
  df_section = pd.DataFrame(index = ts_ev)
  df_rms_plot = pd.DataFrame(index = ts_ev)
  # for every device:
  # gapnans, resample, select event, !find peaks, norm, caluclate rms, plot
  for f in list_devices:
    print('   device: ', dev_n)

    # read data
    df = pd.read_csv(dir_devices + f)

    # gap_nans
    df['datetime_concert'] = (pd.to_datetime(df['timestamp'], unit='ms'))
    df_gapsafe = gap_nans(df, 100)
    df_gapsafe['datetime'] = (pd.to_datetime(df_gapsafe['timestamp'],unit='ms'))

    # resample
    df_resampled = pd.DataFrame(index = ts_dt[:-1])
    cols = df_gapsafe.columns
    for col in cols:
        func = interpolate.interp1d(df_gapsafe['timestamp'], df_gapsafe[col], fill_value='extrapolate')
        df_resampled[col] = func(ts_ts)

    # norm
    aud_df = pd.DataFrame(index = ts_dt[:-1], columns=['time_concert', str(dev_n)])
    aud_df.time_concert = df_resampled.time_concert

    norm = df_resampled.x.diff() # np.linalg.norm(df_resampled[['x','y','z']].diff().values,axis=1)
    
    if np.nanmedian(norm)>0:
        aud_df[str(dev_n)] = norm/np.nanmedian(norm)
    else:
        aud_df[str(dev_n)] = norm/np.nanmean(norm)
    
    # select event
    ## find the indexes with the closest time_concert values
    in_event_index  = aud_df.time_concert.sub(in_event_time).abs().idxmin()
    out_event_index = in_event_index + pd.to_timedelta((ts_ev.shape[0]-1)*20, unit='ms') #aud_df.time_concert.sub(out_event_time).abs().idxmin()
    ## select the indexes
    df_clap = aud_df.loc[in_event_index:out_event_index]

    # low pass filter
    ## parameters
    T = 10.0        # Sample Period
    fs = 50.0       # Hz
    cutoff = 12.5   # cutoff frequency of the filter # Hz
    nyq = 0.5 * fs  # Nyquist Frequency ???
    order = 2 
    n = int(T * fs)
    ## action
    data_raw = df_clap[~isnan(df_clap[str(dev_n)])]
    if data_raw[str(dev_n)].shape[0] > 9:
      data_filtered = butter_lowpass_filter(data_raw[str(dev_n)], cutoff, fs, order)
    else:
      #data_raw[str(dev_n)] = np.nan
      data_filtered = data_raw[str(dev_n)]
    
    df_clap.loc[data_raw.index, 'filtered'] = data_filtered
    
    # rms
    df_rms_signal = rms_signal(df_clap['filtered'], sample_rate, sample_rate)
    df_clap[str(dev_n) + '_rms'] = df_rms_signal

    # load to the df for clapping section
    df_section[str(dev_n)] = df_clap['filtered'].values

    # load data for plot
    df_section_plot = pd.DataFrame(index = df_section.index)
    df_section_plot['rms'] =  df_clap[str(dev_n) + '_rms'].values
    df_section_plot.dropna(inplace=True)
    list_of_lists_x.append(df_section_plot.index)
    list_of_lists_y.append(df_section_plot.rms)
    list_of_device_names.append(f[:3])

    df_rms_plot[str(dev_n) + '_rms'] = df_clap[str(dev_n) + '_rms'].values

    # peaks
    #peak_index_list = scipy.signal.find_peaks(aud_df[str(dev_n)], threshold=1, distance=10)[0] # note: change to filtered
    #df_peaks = aud_df.iloc[peak_index_list]
    #print('peaks: ', df_peaks.shape[0])

    """
    # figure
    title_figure = 'Clapping' + ' - Event: ' + str(section_n).zfill(2) + '_' + ' - at: ' + str(ev_in)
    p = figure(title = title_figure, plot_width=1250, plot_height=300, x_axis_type='datetime', sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
    l0 = p.line(x='index', y=str(dev_n), source=df_clap, line_color='black', legend_label='Accelerometer')
    l1 = p.line(x='index', y=(str(dev_n) + '_rms'), source=df_clap.dropna(), line_color='red', legend_label='RMS')
    #l2 = p.scatter(x='index', y=(str(dev_n)), source=df_peaks, color='red', legend_label='Peaks')
    p.xaxis.ticker.desired_num_ticks = 20
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    plots.append(p)
    """

    # increment counter
    dev_n += 1

    # end devices loop

  # plot section summary - rms
  data_bokeh_rms = pd.DataFrame()
  data_bokeh_rms['xs'] = list_of_lists_x
  data_bokeh_rms['ys'] = list_of_lists_y
  data_bokeh_rms['color'] = (palette[20]*4)[:data_bokeh_rms.shape[0]]

  data_bokeh_signal = pd.DataFrame()
  data_bokeh_signal['xs'] = [df_section.index for i in df_section.columns]
  data_bokeh_signal['ys'] = [df_section[i] for i in df_section.columns]
  data_bokeh_signal['color'] = (palette[20]*4)[:data_bokeh_signal.shape[0]]

  df_rms_plot['avr'] = df_rms_plot.mean(axis=1)

  colors = palette[20]*6
  
  title_figure = ev_tag + '   ' + str(ev_in + extend + shift_concert_time) + ' sec'
  p = figure(title = title_figure, plot_width=2200, plot_height=600, x_axis_type='datetime', sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
  p.multi_line(xs='xs', ys='ys', source=data_bokeh_signal, color='color', legend_label='Signal', line_alpha=0.4)
  p.line(x=df_section.index, y=df_clap[str(dev_n-1)], color='red', legend_label='Signal_raw', line_alpha=0.4) # pg
  for xx, yy, s, i in zip(list_of_lists_x, list_of_lists_y, list_of_device_names, range(len(list_of_lists_x))):
    p.varea(x=xx, y1=0, y2=yy, fill_alpha=0.2, color=colors[i], legend_label='Participants RMS')
  p.line(x='index', y='avr', source=df_rms_plot[df_rms_plot.avr.notna()], line_color = colors[10], line_width=3, legend_label='Average RMS')
  p.title.text_font_size = '20pt' #############
  p.legend.click_policy = 'hide'
  p.legend.orientation = 'horizontal'
  p.add_layout(Title(text='Time', align='center'), 'below')
  p.xaxis.ticker.desired_num_ticks = 20
  p.legend.label_text_font_size = '6pt'
  plots.append(p)

  # save section csv file
  output_dir = 'C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections/'
  output_file_name = 'clapping_' + str(section_n).zfill(2) + '_at_' + str(ev_in + extend + shift_concert_time) + '_sec' + '.csv'
  #df_section.to_csv(output_dir + title_figure.replace(' ', '_') + '.csv')

  # save figure html file
  figure_output_dir = 'C:/ritmo/CopenhagenMusicLab/Concert_content/clapping_sections_plots/'
  #output_file(figure_output_dir + title_figure.replace(' ', '_') + '.html')
  #save(p, figure_output_dir + title_figure.replace(' ', '_') + '.html')

  # increment counter
  section_n += 1

  # end events loop

# show & save all figures in one file
#output_file(figure_output_dir + 'all_clapping_sessions' + '.html')
output_file(figure_output_dir + title_figure.replace(' ', '_') + '_seperate.html')
show(column(plots))

# save figures seperately
#for p in plots:
  #save(p, filename=(figure_output_dir + p.properties_with_values()['title'].text.replace(' ', '_') + '.html'))


1  devices.
time interval:  -692230.0 -673100.0
   device:  0


KeyboardInterrupt: ignored

In [ ]:
df_pg = pd.DataFrame({'Value': [1, np.nan, 3]})
raw_data = df_pg[~np.isnan(df_pg.Value)]
raw_data = raw_data.div(10)
df_pg.loc[raw_data.index, 'Value'] = raw_data.Value.values
pd.DataFrame({'Value': [1, np.nan, 3]}), df_pg

(   Value
 0    1.0
 1    NaN
 2    3.0,
    Value
 0    0.1
 1    NaN
 2    0.3)

In [ ]:
# show & save all figures in one file
#output_file(figure_output_dir + 'all_clapping_sessions' + '.html')
#show(column(plots))

# save figures seperately
for p in plots:
  save(p, filename=(figure_output_dir + p.properties_with_values()['title'].text.replace(' ', '_') + '.html'))


In [ ]:
pl = plots[-1]
span_sync = Span(location=ts_ev[(9701-9679)*50],
                            dimension='height', line_color='olive', line_width=1)
pl.add_layout(span_sync)
source = ColumnDataSource(data=dict(height=[100], weight=[ts_ev[(9701-9679)*50+25]], names=['Synchronous clapping']))
labels = LabelSet(x='weight', y='height', text='names', x_offset=5, y_offset=5, source=source, render_mode='canvas', text_color='olive', text_font_style='italic', text_font='helvetica', angle=math.pi/2)
pl.add_layout(labels)
output_file('final_plot_with_span_iso.html')
save(pl)

ts_ev[(9701-9679)*5]

# pg
---



In [ ]:
# 